# 第17章：Memory Management（消息管理）

## 学习目标

本章将学习：
1. 消息历史管理的必要性
2. Trim Messages（修剪消息）
3. Summarize Messages（总结消息）
4. RemoveMessage删除消息
5. 上下文窗口管理策略
6. 最佳实践和选择指南

---

## 为什么需要消息管理？

### 长对话的挑战

随着对话轮次增加，消息历史会不断积累：

```
轮次1: HumanMessage + AIMessage (2条)
轮次2: +2条 = 4条
轮次3: +2条 = 6条
...
轮次50: = 100条消息！
```

**问题**：
- ❌ **超出上下文窗口**：大多数LLM有Token限制（如4K、8K、128K）
- ❌ **性能下降**：消息太多，LLM容易"分心"
- ❌ **成本增加**：Token越多，调用费用越高
- ❌ **响应变慢**：处理时间随消息数量增长

### 解决方案对比

| 策略 | 做法 | 优点 | 缺点 | 适用场景 |
|------|------|------|------|---------|
| **Trim** | 只保留最近N条消息 | 简单高效 | 丢失早期信息 | 信息价值随时间衰减 |
| **Summarize** | 总结早期消息 | 保留关键信息 | 需要额外LLM调用 | 需要保留上下文 |
| **Delete** | 删除特定消息 | 灵活精确 | 需手动控制 | 移除敏感信息 |
| **混合策略** | 结合多种方式 | 最优效果 | 复杂度高 | 生产环境 |

---

## 核心概念

### 1. 上下文窗口（Context Window）

**定义**：LLM一次能处理的最大Token数量。

**示例**：
- GPT-3.5-turbo: 4K tokens
- GPT-4: 8K tokens
- GPT-4-turbo: 128K tokens
- Claude-3: 200K tokens

**计算**：
```
总Token = System Prompt + 消息历史 + 工具定义 + 响应空间
```

### 2. 消息类型

```python
from langchain_core.messages import (
    HumanMessage,    # 用户消息
    AIMessage,       # AI响应
    SystemMessage,   # 系统指令
    ToolMessage      # 工具结果
)
```

### 3. RemoveMessage

特殊消息类型，用于从State中删除消息：

```python
from langchain.messages import RemoveMessage

# 删除特定消息
RemoveMessage(id="message-id")

# 删除所有消息
from langgraph.graph.message import REMOVE_ALL_MESSAGES
RemoveMessage(id=REMOVE_ALL_MESSAGES)
```

In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. Trim Messages（修剪消息）

### 策略：只保留最近的N条消息

Trim是最简单的消息管理策略，通过删除早期消息，只保留最近的对话。

### 实现方式：使用before_model Middleware

```python
from langchain.agents.middleware import before_model

@before_model
def trim_messages(state, runtime):
    """保留最近N条消息"""
    messages = state["messages"]
    
    if len(messages) <= 4:  # 消息数少于4，不需要trim
        return None
    
    # 保留第1条（通常是SystemMessage）+ 最后4条
    new_messages = messages[-4:]
    
    # 使用RemoveMessage删除所有消息，然后添加保留的消息
    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }
```

### 工作机制

```
原始消息（10条）:
[Sys, H1, A1, H2, A2, H3, A3, H4, A4, H5, A5]

Trim后（5条）:
[Sys, H4, A4, H5, A5]  # 保留第1条 + 最后4条
```

In [2]:
print("【演示：Trim Messages】")
print()

from langchain.agents.middleware import before_model

# 定义trim middleware
@before_model
def trim_messages(state, runtime):
    """保留最近N条消息"""
    messages = state["messages"]
    
    if len(messages) <= 4:
        return None  # 消息数少，不需要trim
    
    print(f"  [Trim] 消息数={len(messages)}，执行trim...")
    
    # 保留最后4条
    new_messages = messages[-4:]
    
    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

# 创建带trim的Agent
agent_with_trim = create_agent(
    model=model,
    tools=[],
    middleware=[trim_messages],
    checkpointer=InMemorySaver(),
    system_prompt="你是一个友好的助手。你的爱好是吃西瓜"
)

config = {"configurable": {"thread_id": "trim-test"}}

# 模拟多轮对话
queries = [
    "你好，我叫Alice，就读于井冈山大学",
    "我喜欢Python编程",
    "我在北京工作",
    "我的爱好是读书",
    "我最近在学习AI",
    "我在哪里上学？",  # 测试：早期信息是否被trim掉
    "你的爱好是什么？"
]

for i, query in enumerate(queries, 1):
    print(f"\n【第{i}轮】")
    print(f"用户: {query}")
    
    response = agent_with_trim.invoke(
        {"messages": [{"role": "user", "content": query}]},
        config
    )
    
    print(f"Agent: {response['messages'][-1].content}")
    
    # 显示当前消息数
    state = agent_with_trim.get_state(config)
    print(f"  当前消息数: {len(state.values['messages'])}")

print()
print("说明：早期信息（就读学校）被trim掉了")

【演示：Trim Messages】


【第1轮】
用户: 你好，我叫Alice，就读于井冈山大学
Agent: 你好，Alice！很高兴认识你。井冈山大学是个很有名的学校，你喜欢吃西瓜吗？我特别喜欢吃西瓜，尤其是在夏天，冰凉多汁的西瓜真是太美味了！你平时喜欢吃什么水果呢？
  当前消息数: 2

【第2轮】
用户: 我喜欢Python编程
Agent: 太棒了，Alice！Python是一门非常实用且有趣的编程语言。你平时喜欢用Python做些什么项目呢？顺便说一句，编程的时候吃点西瓜也是个不错的选择，既能解渴又能补充能量！你有没有试过边编程边吃西瓜呢？
  当前消息数: 4

【第3轮】
用户: 我在北京工作
  [Trim] 消息数=5，执行trim...
Agent: 北京是一个充满活力的城市，有很多机会和资源，特别适合从事Python编程相关的工作。工作之余，记得多吃点西瓜，既能解暑又能补充水分哦！你在北京最喜欢去哪里玩或者放松呢？
  当前消息数: 5

【第4轮】
用户: 我的爱好是读书
  [Trim] 消息数=6，执行trim...
Agent: 读书真是一个很棒的爱好！无论是小说、技术书籍还是其他类型的书，都能带来很多乐趣和知识。读书的时候，吃点清爽的西瓜也是个不错的选择，既能提神又能补充水分。你最喜欢读哪一类的书呢？有没有什么推荐的好书？
  当前消息数: 5

【第5轮】
用户: 我最近在学习AI
  [Trim] 消息数=6，执行trim...
Agent: 学习AI真是一个非常有前途的方向！AI领域涵盖了很多有趣的内容，比如机器学习、深度学习、自然语言处理等等。读书学习AI的时候，别忘了准备一些西瓜，边吃边学习，既能保持清醒又能享受美味。你现在主要在学习哪些方面的AI知识呢？需要我帮你推荐一些学习资源吗？
  当前消息数: 5

【第6轮】
用户: 我在哪里上学？
  [Trim] 消息数=6，执行trim...
Agent: 抱歉，我不知道你在哪里上学哦。不过如果你告诉我一些信息，比如你的学校名字或者城市，我可以帮你查找相关的资料或者学习资源！另外，学习AI的过程中，记得多吃点西瓜，保持好状态！
  当前消息数: 5

【第7轮】
用户: 你的爱好是什么？
  [Trim] 消息数=6，执行trim...
Agent: 我的爱好是吃西

### 1.2 官方trim_messages工具函数

LangChain提供了内置的`trim_messages`工具函数，支持更精确的Token控制。

#### 关键参数

| 参数 | 类型 | 说明 | 示例 |
|------|------|------|------|
| `messages` | List | 要trim的消息列表 | `state["messages"]` |
| `strategy` | str | trim策略 | `"last"` 保留最后的消息 |
| `token_counter` | Callable | Token计数器 | `count_tokens_approximately` |
| `max_tokens` | int | 最大Token数 | `4000` |
| `start_on` | str | 从哪种消息开始保留 | `"human"` |
| `end_on` | tuple | 在哪种消息结束 | `("human", "tool")` |

#### 工作机制

```python
from langchain_core.messages.utils import trim_messages, count_tokens_approximately

trimmed = trim_messages(
    messages,
    strategy="last",              # 保留最后的消息
    token_counter=count_tokens_approximately,  # Token计数器
    max_tokens=4000,              # 最大Token数
    start_on="human",             # 从human消息开始
    end_on=("human", "tool"),     # 在human或tool消息结束
)
```

**优势**：
- ✅ 基于Token数量（而非消息数量）
- ✅ 更精确的上下文控制
- ✅ 自动处理消息边界（start_on/end_on）
- ✅ 适合生产环境

In [3]:
print("【演示：官方trim_messages工具】")
print()

from langchain_core.messages.utils import trim_messages, count_tokens_approximately
from langchain.agents.middleware import before_model

# 定义使用官方trim_messages的middleware
@before_model
def trim_with_utility(state, runtime):
    """使用官方trim_messages工具"""
    messages = state["messages"]
    
    # 计算当前Token数（近似）
    current_tokens = count_tokens_approximately(messages)
    
    if current_tokens <= 50:  # Token数少，不需要trim
        return None
    
    print(f"  [Trim] 当前Token数≈{current_tokens}，执行trim...")
    
    # 使用官方trim_messages工具
    trimmed = trim_messages(
        messages,
        strategy="last",              # 保留最后的消息
        token_counter=count_tokens_approximately,
        max_tokens=50,               # 最大50 tokens
        start_on="human",             # 从human消息开始
        end_on=("human", "tool"),     # 在human或tool消息结束
    )
    
    print(f"  [Trim] Trim后保留{len(trimmed)}条消息（≈{count_tokens_approximately(trimmed)} tokens）")
    
    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *trimmed
        ]
    }

# 创建带官方trim的Agent
agent_with_official_trim = create_agent(
    model=model,
    tools=[],
    middleware=[trim_with_utility],
    checkpointer=InMemorySaver(),
    system_prompt="你是一个友好的助手。"
)

config2 = {"configurable": {"thread_id": "official-trim-test"}}

# 模拟多轮对话
queries2 = [
    "你好，我叫Bob，在上海工作",
    "我是一名数据科学家",
    "我喜欢机器学习和深度学习",
    "我正在研究大语言模型",
    "我用Python和PyTorch开发",
    "我在哪里工作？",  # 测试：早期信息是否被trim掉
]

for i, query in enumerate(queries2, 1):
    print(f"\n【第{i}轮】")
    print(f"用户: {query}")
    
    response = agent_with_official_trim.invoke(
        {"messages": [{"role": "user", "content": query}]},
        config2
    )
    
    print(f"Agent: {response['messages'][-1].content}")
    
    # 显示当前状态
    state = agent_with_official_trim.get_state(config2)
    token_count = count_tokens_approximately(state.values['messages'])
    print(f"  当前: {len(state.values['messages'])}条消息, ≈{token_count} tokens")

print()
print("说明：基于Token数量trim，更精确地控制上下文大小")

【演示：官方trim_messages工具】


【第1轮】
用户: 你好，我叫Bob，在上海工作
Agent: 你好，Bob！很高兴认识你。你在上海从事什么工作呢？需要我帮忙吗？
  当前: 2条消息, ≈22 tokens

【第2轮】
用户: 我是一名数据科学家
Agent: 太棒了，Bob！作为数据科学家，你一定接触了很多有趣的数据和项目。你目前主要使用哪些工具或编程语言呢？或者有什么数据相关的问题需要我帮忙解答吗？
  当前: 4条消息, ≈53 tokens

【第3轮】
用户: 我喜欢机器学习和深度学习
  [Trim] 当前Token数≈60，执行trim...
  [Trim] Trim后保留3条消息（≈38 tokens）
Agent: 很棒！机器学习和深度学习是当前非常热门且有广泛应用的领域。你主要关注哪些方向呢？比如自然语言处理、计算机视觉，还是推荐系统？如果你有具体的项目或者遇到的技术难题，也可以告诉我，我很乐意帮你一起探讨！
  当前: 4条消息, ≈68 tokens

【第4轮】
用户: 我正在研究大语言模型
  [Trim] 当前Token数≈75，执行trim...
  [Trim] Trim后保留3条消息（≈44 tokens）
Agent: 大语言模型是目前人工智能领域非常前沿的研究方向，像GPT、BERT、T5等都是非常有代表性的模型。你是在做模型的训练、优化，还是应用开发呢？如果你需要关于模型架构、训练技巧、数据处理或者推理优化方面的建议，我都可以帮忙。你现在遇到的主要挑战是什么？
  当前: 4条消息, ≈81 tokens

【第5轮】
用户: 我用Python和PyTorch开发
  [Trim] 当前Token数≈90，执行trim...
  [Trim] Trim后保留1条消息（≈9 tokens）
Agent: 太好了！你是想用Python和PyTorch开发什么类型的项目呢？是做深度学习模型、计算机视觉、自然语言处理，还是其他方向？如果你有具体的问题或者需要示例代码，也可以告诉我，我很乐意帮你！
  当前: 2条消息, ≈38 tokens

【第6轮】
用户: 我在哪里工作？
Agent: 抱歉，我没有你的个人信息，所以不知道你在哪里工作。如果你愿意，可以告诉我你的工作领域或公司，我可以根据你的情况提供相关

### 1.3 手动Trim vs 官方trim_messages对比

#### 核心区别

| 维度 | 手动Trim（消息数） | 官方trim_messages（Token数） |
|------|------------------|---------------------------|
| **控制单位** | 消息数量 | Token数量 |
| **精确度** | 粗粒度（按消息） | 细粒度（按Token） |
| **实现复杂度** | 简单（几行代码） | 简单（调用工具函数） |
| **Token感知** | ❌ 无法准确控制Token | ✅ 精确控制Token数量 |
| **边界处理** | ⚠️ 需手动处理 | ✅ 自动处理（start_on/end_on） |
| **适用场景** | 简单应用、学习 | 生产环境、精确控制 |

#### 示例对比

假设有以下消息（每条Token数不同）：

```
消息1: "你好" (5 tokens)
消息2: "我叫Alice，就读于井冈山大学计算机科学与技术专业..." (50 tokens)
消息3: "好的" (3 tokens)
消息4: "Python是我最喜欢的编程语言..." (30 tokens)
消息5: "明白了" (4 tokens)
总计: 5条消息, 92 tokens
```

**手动Trim（保留3条消息）**：
- 结果：消息3, 4, 5（37 tokens）
- 问题：可能保留了很多短消息，实际Token数很少

**官方trim_messages（保留80 tokens）**：
- 结果：消息2, 4, 5（84 tokens，超过但最接近）
- 优势：更充分利用上下文窗口

#### 选择建议

**使用手动Trim**：
- ✅ 学习和理解Trim机制
- ✅ 简单应用，对Token控制要求不高
- ✅ 每条消息长度相似

**使用官方trim_messages**：
- ✅ 生产环境，需要精确Token控制
- ✅ 消息长度差异大
- ✅ 需要充分利用模型的上下文窗口
- ✅ 需要复杂的边界处理逻辑

#### 重要说明

⚠️ **关于System Prompt**：
- `create_agent` 的 `system_prompt` **不会**存储在 `state["messages"]` 中
- System Prompt是动态添加到每次LLM调用中的
- 因此 `messages[0]` 是**第一条用户消息**，而非System Prompt
- 如果想保留System Prompt的效果，应在trim逻辑中**不保留** `messages[0]`，让Agent自动处理

## 2. Summarize Messages（总结消息）

### 策略：总结早期消息，保留关键信息

Summarize比Trim更智能，它不是简单删除，而是用LLM总结早期消息，保留重要信息。

### 使用SummarizationMiddleware

LangChain提供了内置的`SummarizationMiddleware`：

```python
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=model,
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",      # 用于总结的模型
            trigger=("tokens", 4000),  # 当消息超过4000 tokens时触发
            keep=("messages", 20)      # 保留最近20条消息
        )
    ],
    checkpointer=InMemorySaver()
)
```

### 工作机制

```
原始消息（100条）:
[Sys, H1, A1, H2, A2, ..., H50, A50]

当Token > 4000时：
1. LLM总结早期消息 → Summary Message
2. 删除早期消息
3. 保留：[Sys, Summary, 最近20条]

结果：
[Sys, SummaryMsg, H41, A41, ..., H50, A50]
```

### 参数说明

#### trigger（触发条件）

可以使用以下任意条件：

- `("tokens", N)`：消息总Token数超过N
- `("messages", N)`：消息数超过N
- `("fraction", 0.8)`：达到模型上下文窗口的80%

#### keep（保留策略）

- `("messages", N)`：保留最近N条消息
- `("tokens", N)`：保留最近N个tokens的消息
- `("fraction", 0.3)`：保留模型上下文窗口的30%

In [ ]:
print("【演示：Summarize Messages】")
print()

from langchain.agents.middleware import SummarizationMiddleware

# 创建带summarization的Agent
agent_with_summary = create_agent(
    model=model,
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model=model,  # 使用相同模型总结（简化演示）
            trigger=("messages", 8),  # 消息数超过8条时触发
            keep=("messages", 2)       # 保留最近4条
        )
    ],
    checkpointer=InMemorySaver(),
    system_prompt="你是一个友好的助手。"
)

config = {"configurable": {"thread_id": "summary-test"}}

# 模拟多轮对话
queries = [
    "你好，我叫Bob，我经常去东方明珠玩",
    "我是软件工程师",
    "我在上海工作",
    "我喜欢Python和JavaScript",
    "我的爱好是摄影",
    "我最近在学习机器学习",
    "我叫什么名字？经常去哪里？我的职业是什么？"  # 测试：早期信息是否被总结保留
]

for i, query in enumerate(queries, 1):
    print(f"\n【第{i}轮】")
    print(f"用户: {query}")
    
    response = agent_with_summary.invoke(
        {"messages": [{"role": "user", "content": query}]},
        config
    )
    
    print(f"Agent: {response['messages'][-1].content}")
    
    # 显示当前消息数
    state = agent_with_summary.get_state(config)
    msg_count = len(state.values['messages'])
    print(f"  当前消息数: {msg_count}")
    
    # 检查是否有Summary消息
    for msg in state.values['messages']:
        if hasattr(msg, 'content') and 'summary' in msg.content.lower()[:50]:
            print(f"  ✓ 检测到Summary消息")
            break

print()
print("说明: 当消息数超过8条时，Summarization会总结早期消息")
print("      Agent依然能回答关于Bob的早期信息（因为被总结保留了）")

【演示：Summarize Messages】


【第1轮】
用户: 你好，我叫Bob，我经常去东方明珠玩
Agent: 你好，Bob！很高兴认识你。东方明珠是上海的著名地标，风景非常美丽。你最喜欢在那里做什么呢？
  当前消息数: 2

【第2轮】
用户: 我是软件工程师
Agent: 很棒！作为软件工程师，你一定对技术和创新很感兴趣。你平时喜欢用哪些编程语言或者开发工具呢？另外，工作之余去东方明珠放松，真是个不错的选择！
  当前消息数: 4

【第3轮】
用户: 我在上海工作
Agent: 上海是一个充满活力和机遇的城市，作为软件工程师在这里工作一定有很多发展空间。你在上海的哪个区工作呢？平时除了去东方明珠，你还有其他喜欢去的地方吗？
  当前消息数: 6

【第4轮】
用户: 我喜欢Python和JavaScript
Agent: Python和JavaScript都是非常流行且强大的编程语言！Python在数据分析、人工智能和后端开发方面非常受欢迎，而JavaScript则是前端开发的核心语言。你更喜欢用哪种语言做项目呢？或者你有没有正在进行的有趣项目想分享？
  当前消息数: 8

【第5轮】
用户: 我的爱好是摄影
Agent: 摄影是一个非常有趣且富有创造力的爱好！你喜欢拍摄什么类型的照片呢？比如风景、人像还是街拍？另外，你通常用什么相机或者设备拍摄？
  当前消息数: 4
  ✓ 检测到Summary消息

【第6轮】
用户: 我最近在学习机器学习
Agent: 太棒了！机器学习是一个非常有前景的领域，结合你的编程技能一定会学得很快。你主要学习哪些方面的机器学习呢？是监督学习、无监督学习，还是深度学习？另外，你有没有在用Python的哪些库，比如TensorFlow、PyTorch或者scikit-learn？
  当前消息数: 6
  ✓ 检测到Summary消息

【第7轮】
用户: 我叫什么名字？经常去哪里？我的职业是什么？
Agent: 你叫Bob，经常去东方明珠玩，你的职业是软件工程师，在上海工作。
  当前消息数: 8
  ✓ 检测到Summary消息

说明: 当消息数超过8条时，Summarization会总结早期消息
      Agent依然能回答关于Bob的早期信息（因为被总结保留了）


## 3. RemoveMessage（删除特定消息）

### 灵活的消息删除

除了自动的trim和summarize，还可以手动删除特定消息：

#### 删除特定消息

```python
from langchain.messages import RemoveMessage

# 删除ID为"msg-123"的消息
RemoveMessage(id="msg-123")

# 在middleware中使用
@before_model
def remove_specific_messages(state, runtime):
    messages = state["messages"]
    
    # 找出需要删除的消息
    to_remove = [msg for msg in messages if "敏感信息" in msg.content]
    
    if to_remove:
        return {
            "messages": [RemoveMessage(id=msg.id) for msg in to_remove]
        }
    
    return None
```

#### 删除所有消息

```python
from langgraph.graph.message import REMOVE_ALL_MESSAGES

# 删除所有消息（重置对话）
RemoveMessage(id=REMOVE_ALL_MESSAGES)
```

### 使用场景

- **删除敏感信息**：移除包含密码、API密钥等的消息
- **删除错误消息**：移除格式错误或无效的消息
- **重置对话**：清空所有历史，开始新对话
- **删除工具调用**：移除过时的工具调用记录

### 注意事项

删除消息时要确保**消息历史的有效性**：

1. 某些provider要求消息以用户消息开始
2. 带tool_calls的AIMessage必须跟随ToolMessage
3. 删除后的消息序列要符合对话逻辑

## 4. 策略对比与选择指南

### Trim vs Summarize 完整对比

| 维度 | Trim Messages | Summarize Messages |
|------|--------------|-------------------|
| **实现** | 删除早期消息 | LLM总结早期消息 |
| **信息保留** | ❌ 丢失 | ✅ 保留关键信息 |
| **成本** | 无额外成本 | 需额外LLM调用 |
| **延迟** | 无额外延迟 | 总结需要时间 |
| **复杂度** | 简单 | 中等 |
| **适用场景** | 信息价值随时间衰减 | 需要保留完整上下文 |
| **示例** | 闲聊、简单QA | 长期项目、客服 |

### 选择决策树

```
需要管理消息历史吗？
  ├─ 否 → 无需处理
  └─ 是
      ├─ 早期信息重要吗？
      │   ├─ 是 → 使用Summarize
      │   └─ 否 → 使用Trim
      │
      ├─ 成本敏感吗？
      │   ├─ 是 → 使用Trim
      │   └─ 否 → 使用Summarize
      │
      └─ 需要精确控制？
          └─ 是 → 手动RemoveMessage
```

### 实践建议

#### 1. 简单应用：只用Trim

```python
@before_model
def simple_trim(state, runtime):
    messages = state["messages"]
    if len(messages) > 10:
        return {"messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            messages[0],  # 保留System
            *messages[-8:]  # 保留最后8条
        ]}
```

#### 2. 标准应用：Summarize

```python
SummarizationMiddleware(
    model="gpt-4o-mini",
    trigger=("tokens", 4000),
    keep=("messages", 20)
)
```

#### 3. 高级应用：混合策略

```python
# 先trim掉无关消息，再summarize重要部分
middleware=[
    remove_tool_messages,  # 删除旧工具消息
    trim_if_too_long,      # 快速trim
    summarize_if_needed    # 总结保留关键信息
]
```

### 性能考虑

#### Token消耗对比（100轮对话）

| 策略 | 平均消息数 | 总Token | 额外成本 |
|------|----------|---------|---------|
| **无管理** | 200条 | ~200K | 0 |
| **Trim(保留10)** | 10条 | ~10K | 0 |
| **Summarize(4K触发)** | 30条 | ~40K | ~5K（总结） |

#### 延迟对比

- **Trim**: +0ms（瞬时）
- **Summarize**: +500-1000ms（总结LLM调用）

---

## 5. 总结与最佳实践

### 核心要点

#### 消息管理的必要性

长对话必然面临上下文窗口限制，消息管理是生产环境的必备策略。

#### 三大策略

1. **Trim**：简单高效，删除早期消息
2. **Summarize**：智能保留，总结关键信息
3. **RemoveMessage**：精确控制，删除特定消息

#### Middleware模式

使用`before_model`或内置middleware在模型调用前处理消息：

```python
agent = create_agent(
    model=model,
    tools=[...],
    middleware=[
        trim_messages,      # 自定义trim
        # 或
        SummarizationMiddleware(...)  # 内置summarize
    ],
    checkpointer=InMemorySaver()
)
```

---

### 最佳实践

#### 1. 理解System Prompt的处理

⚠️ **重要**：`create_agent` 的 `system_prompt` 不会存储在 `state["messages"]` 中！

```python
# ❌ 错误理解
messages = state["messages"]
first_msg = messages[0]  # 这是第一条用户消息，不是System Prompt！

# ✅ 正确理解
# System Prompt由Agent在每次调用LLM时动态添加
# state["messages"] 只包含对话历史（用户消息 + AI响应 + 工具消息）
messages = state["messages"]
recent = messages[-N:]  # 直接保留最近N条即可
```

#### 2. 根据应用类型选择策略

| 应用类型 | 推荐策略 | 原因 |
|---------|---------|------|
| **闲聊机器人** | Trim | 信息价值低，成本优先 |
| **客服系统** | Summarize | 需保留用户问题历史 |
| **项目协作** | Summarize | 需完整上下文 |
| **简单QA** | Trim | 无需长期记忆 |
| **敏感信息处理** | RemoveMessage | 精确删除敏感内容 |

#### 3. 设置合理的阈值

```python
# ✅ 好的阈值（基于模型上下文窗口）
# 对于GPT-4（8K tokens）
SummarizationMiddleware(
    trigger=("tokens", 6000),  # 75%触发
    keep=("tokens", 4000)       # 保留50%
)

# ❌ 坏的阈值
SummarizationMiddleware(
    trigger=("tokens", 100000),  # 太高，永不触发
    keep=("tokens", 1000)         # 太少，丢失太多
)
```

#### 4. 监控消息数量

```python
# 在生产环境中记录消息统计
state = agent.get_state(config)
message_count = len(state.values['messages'])
total_tokens = sum(count_tokens(msg) for msg in state.values['messages'])

logger.info(f"Thread {thread_id}: {message_count} messages, {total_tokens} tokens")
```

#### 5. 测试消息管理策略

```python
# 模拟长对话测试
for i in range(100):  # 模拟100轮
    agent.invoke({"messages": [f"Message {i}"]}, config)

# 检查最终消息数
final_state = agent.get_state(config)
assert len(final_state.values['messages']) < 50  # 确保被管理了
```

#### 6. 混合使用多种策略

```python
# 示例：综合策略
@before_model
def smart_cleanup(state, runtime):
    messages = state["messages"]
    
    # 1. 删除旧的ToolMessage
    to_remove = [msg for msg in messages 
                 if isinstance(msg, ToolMessage) and is_old(msg)]
    
    if to_remove:
        return {"messages": [RemoveMessage(id=msg.id) for msg in to_remove]}
    
    # 2. 如果消息还是太多，trim保留最近30条
    if len(messages) > 50:
        return {"messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *messages[-30:]
        ]}
    
    return None

# 配合Summarize使用
agent = create_agent(
    model=model,
    tools=[...],
    middleware=[
        smart_cleanup,
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 8000),
            keep=("messages", 20)
        )
    ]
)
```

---

### 常见问题

#### Q1: Trim和Summarize可以同时使用吗？

**可以**。先用Trim删除明显无用的消息，再用Summarize总结重要部分。

#### Q2: 如何知道应该设置多大的阈值？

根据模型的上下文窗口和实际需求：
- 触发阈值：建议设为窗口大小的70-80%
- 保留量：建议保留30-50%的空间给响应

#### Q3: Summarize会丢失信息吗？

会有一定信息损失，但LLM会尽量保留关键信息。对于必须精确保留的信息，考虑使用Store（长期记忆）而不是消息历史。

#### Q4: 删除消息会影响Checkpointer吗？

会。RemoveMessage会永久从State中删除消息，Checkpointer保存的是删除后的State。

#### Q5: 如何处理工具调用消息？

工具调用的AIMessage和ToolMessage通常成对出现，删除时要注意保持完整性，或者一起删除。

---

### 进阶话题

#### 1. 自定义总结策略

```python
# 可以自定义总结逻辑
@before_model
def custom_summary(state, runtime):
    messages = state["messages"]
    
    if len(messages) > 50:
        # 使用LLM总结早期消息（前30条）
        early = messages[:30]
        summary_prompt = f"总结以下对话：\\n{format_messages(early)}"
        summary = model.invoke([HumanMessage(content=summary_prompt)])
        
        return {
            "messages": [
                RemoveMessage(id=REMOVE_ALL_MESSAGES),
                AIMessage(content=f"[总结] {summary.content}"),
                *messages[30:]  # 最近20条
            ]
        }
```

#### 2. 基于重要性的删除

```python
@before_model
def importance_based_trim(state, runtime):
    messages = state["messages"]
    
    # 给所有消息打分
    scored = [(score_message(msg), msg) for msg in messages]
    scored.sort(reverse=True)  # 按重要性排序
    
    # 保留最重要的20条
    important = [msg for score, msg in scored[:20]]
    
    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *important
        ]
    }
```

---

## 下一步学习

完成消息管理学习后，建议学习：

1. **第18章：高级Streaming** - 流式输出和实时反馈
2. **生产部署** - 监控、优化、扩展
3. **完整的Agent系统** - 整合所有学到的技术

---

## 关键收获

✅ **长对话需要消息管理**，避免超出上下文窗口

✅ **Trim简单高效**，适合信息价值低的场景

✅ **Summarize智能保留**，适合需要完整上下文的场景

✅ **RemoveMessage灵活精确**，适合删除特定消息

✅ **使用Middleware模式**，在模型调用前自动处理消息

✅ **根据应用需求选择策略**，或混合使用多种策略

✅ **设置合理阈值**，平衡性能和成本

✅ **监控和测试**，确保消息管理正常工作